In [ ]:
# Import all needed libraries
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from scipy.stats import uniform, randint

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Authorize access to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read CSV files into dataframes
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/ML Project/Layer 9/train.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/ML Project/Layer 9/valid.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/ML Project/Layer 9/test.csv')

In [ ]:
train_df.shape

(28520, 772)

In [ ]:
train_df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.019301,0.059756,0.081375,0.057481,-0.068440,-0.165913,0.035643,-0.091138,0.021688,0.057158,...,-0.035576,0.127319,0.098128,-0.058787,0.100971,-0.047754,45,NaN,1,6
1,0.049741,0.090030,0.035118,-0.013676,-0.194317,-0.101763,0.085875,-0.081317,0.112418,0.120523,...,0.020538,0.058968,0.029803,0.111324,0.036727,0.031927,45,NaN,1,6
2,0.019212,0.087779,0.093907,-0.033738,-0.141409,-0.062881,-0.071402,-0.006599,0.020372,-0.027777,...,0.119645,-0.040861,0.000548,-0.061003,-0.042450,0.063340,45,NaN,1,6
3,0.070283,0.049040,0.042126,0.122637,-0.056964,-0.113700,0.108454,0.051336,0.086610,0.141578,...,-0.124494,-0.169225,-0.046391,0.148787,0.014616,-0.140644,45,NaN,1,6
4,0.028864,0.165634,0.016302,0.036117,-0.028871,-0.147748,0.053180,0.025071,-0.004200,-0.022183,...,-0.124862,0.044907,0.084005,-0.038450,0.084371,-0.072146,45,NaN,1,6


## **Data Preprocessing**

In [ ]:
# Identify columns with null values and count how many missing values each of those columns has.
missing_columns = train_df.columns[train_df.isnull().any()]
missing_counts = train_df[missing_columns].isnull().sum()

print('Missing Columns and Number of Counts')
for column in missing_columns:
    print( str(column) +' : '+ str(missing_counts[column]))

Missing Columns and Number of Counts
label_2 : 480


In [ ]:
# Create new copies of the dataframes
train_data = train_df.copy()
valid_data = valid_df.copy()
test_data = test_df.copy()

In [ ]:
train_df.describe()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
count,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,...,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28040.000000,28520.000000,28520.000000
mean,-0.015964,0.075453,0.045892,0.046230,-0.068548,-0.090882,0.034405,-0.038091,0.049623,0.036286,...,-0.058015,0.017027,-0.002970,0.053544,0.033636,0.003864,30.498843,27.975107,0.799299,5.997125
std,0.048122,0.047099,0.059531,0.056354,0.062569,0.054895,0.065229,0.058487,0.051186,0.060626,...,0.075779,0.055990,0.055413,0.082085,0.070793,0.059617,17.328389,5.735913,0.400532,2.375567
min,-0.300831,-0.113586,-0.183329,-0.212180,-0.385634,-0.331156,-0.320988,-0.519168,-0.171353,-0.404408,...,-0.343703,-0.401326,-0.355695,-0.390809,-0.470045,-0.242932,1.000000,22.000000,0.000000,0.000000
25%,-0.048373,0.046609,0.004613,0.007529,-0.109321,-0.128968,0.001598,-0.073965,0.015015,-0.001981,...,-0.105128,-0.014950,-0.039802,0.003270,-0.006049,-0.038369,15.000000,25.000000,1.000000,6.000000
50%,-0.017313,0.073738,0.049874,0.044959,-0.068874,-0.089224,0.039623,-0.034788,0.050111,0.036198,...,-0.069593,0.021222,-0.003279,0.057083,0.036988,0.004224,30.000000,27.000000,1.000000,6.000000
75%,0.015003,0.101526,0.087481,0.082385,-0.026185,-0.052010,0.076055,0.002081,0.085094,0.074090,...,-0.028467,0.053432,0.033408,0.109684,0.078949,0.044731,46.000000,30.000000,1.000000,6.000000
max,0.316810,0.519338,0.344814,0.443703,0.169458,0.298595,0.309478,0.246881,0.260366,0.270373,...,0.388045,0.215233,0.205440,0.332756,0.269439,0.385349,60.000000,61.000000,1.000000,13.000000


Only a small number of missing values **(480)** are found in the dataset compared to the total number of rows **(28520)**. Therefore, these missing rows are decided to be **removed** from the dataset, since their **impact is much smaller and negligible**

In [ ]:
x_train = {}
x_valid = {}
x_test = {}

y_train = {}
y_valid = {}
y_test = {}

# Create dictionaries for each of the labels
for target_label in ['label_1','label_2','label_3','label_4']:

  # Select only rows with non-null values
  if target_label == "label_2":
    train = train_df[train_df['label_2'].notna()]
    valid = valid_df[valid_df['label_2'].notna()]

  else:
    train = train_df
    valid = valid_df

  test = test_df

  # A data preprocessing technique to scale and transform dataset features while minimizing sensitivity to outliers.
  scaler = RobustScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(train.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_train[target_label] = train[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(valid.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_valid  [target_label] = valid[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(test.drop(["ID"],axis=1)), columns=[f'feature_{i}' for i in range(1,769)])

# **Label 01**

In [ ]:
# Create new copies
x_train_df = x_train['label_1'].copy()
y_train_df = y_train['label_1'].copy()

x_valid_df = x_valid['label_1'].copy()
y_valid_df = y_valid['label_1'].copy()

x_test_df = x_test['label_1'].copy()

## **K-Fold Cross-Validation Approach**
This technique is used to assess machine learning model performance by dividing the dataset into K subsets, training the model K times, and averaging the results. It helps reduce the impact of data splitting variability, maximize data utilization, and useful for model selection.

In [ ]:
# Perform 5-fold cross-validation Process
scores = cross_val_score(SVC(), x_train_df, y_train_df, cv=5, scoring='accuracy')

mean_accuracy = scores.mean()
std_accuracy = scores.std()

# Print the obtained cross-validation scores
print('Support Vector Machines')
print('\n')
print("Cross-validation scores: ", scores)
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")

Support Vector Machines


Cross-validation scores:  [0.93758766 0.94214586 0.93793829 0.93653576 0.94074334]
Mean Accuracy: 0.94
Standard Deviation: 0.00


## **Feature Selection - Principal Component Analysis (PCA)**
This is a common technique for feature selection and visualization in machine learning, helping to identify and focus on the most important patterns and relationships within the data.

This technique is applied here to improve the performance of the model by increasing its accuracy score. The number of best features obtained here is **430**.

In [ ]:
# Perform Principal Component Analysis (PCA) Approach
pca = PCA(n_components=0.975, svd_solver='full')
pca.fit(x_train_df)

# Create new dataframes by performing PCA transformation
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df))
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df))
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df))

# Print the obtained shape after applying PCA approach
print('Shape after PCA: ',x_train_df_pca.shape)

Shape after PCA:  (28520, 430)


# **Build Classifier Models**
Both Support Vector Classifier model and Random Forest Classifier model are built, optimized and evaluated based on the accuracy score obtained to choose the best model

## **Support Vector Machine (SVM)**
Support Vector Machines (SVM) are effective for both linear and non-linear classification tasks. Support Vector Classification (SVC) is a variant of SVM used for classification tasks, where the goal is to separate data points into different classes using a hyperplane while maximizing the margin between the classes.

SVC model is built for the dataframe obtained after PCA approach. The accuracy score obtained here **(0.9453)** is **higher** than the accuracy score obtained on initial dataframe before feature selection **(0.94)**. So, **feature selection have improved the performance of the classifier model.**

In [ ]:
# Perform Support Vector classification (SVC) model approach
classifier = SVC(kernel='linear', C=1)
classifier.fit(x_train_df_pca, y_train_df)
y_valid_pred = classifier.predict(x_valid_df_pca)

# Print the obtained accuracy score after applying SVC approach
print("Accuracy Score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

Accuracy Score:  0.9453333333333334


## **Hyperparameter Tuning**
This approach is used to optimize a machine learning model's performance by finding the best set of hyperparameters. These hyperparameters control the behavior of the model and fine-tuning them can significantly impact the model's accuracy and generalization to new data.

In [ ]:
# Dictionary that defines hyperparameters list to search over during tuning process.
param_dist = {
    'C': [100,10,1,0,0.1,0.01],
    'kernel': ['rbf','linear','poly','sigmoid'],
    'gamma': ['scale','auto'],
    'degree': [1,2,3,4],
    'class_weight' : ['none','balanced']
}

svm = SVC()

# Perform randomized hyperparameter search
random_search = RandomizedSearchCV(
    svm, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42, scoring='accuracy'
)
random_search.fit(x_train_df_pca, y_train_df)

# Retrieve the best hyperparameters obtained.
best_params = random_search.best_params_
best_model = random_search.best_estimator_
print("Best Parameters: ", best_params)

Best Parameters:  {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'class_weight': 'balanced', 'C': 100}


**K-Fold Cross-Validation** Approach for dataframe obtained after **Principal Component Analysis (PCA)** approach

In [ ]:
# Perform 5-fold cross-validation Process
scores = cross_val_score(classifier, x_train_df_pca, y_train_df, cv=5, scoring='accuracy')

mean_accuracy = scores.mean()
std_accuracy = scores.std()

# Print the obtained cross-validation scores
print('Support Vector Machines')
print('\n')
print("Cross-validation scores: ", scores)
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")

Support Vector Machines


Cross-validation scores:  [0.94267181 0.9437237  0.94091865 0.9391655  0.94582749]
Mean Accuracy: 0.94
Standard Deviation: 0.00


## **Support Vector Classifier** model with **Best Hyper Parameters**
This is done to build a classification model that is fine-tuned for the specific problem. The use of obtained **hyper parameters** have improved the accuracy score **(0.9586)** by a significant value compared to before obtained accuracy score **(0.9453)**.

In [ ]:
# Perform Support Vector Classification (SVC) model
classifier = SVC(kernel='rbf', gamma='scale', degree=4, class_weight='balanced', C=100)
classifier.fit(x_train_df_pca, y_train_df)
y_valid_pred = classifier.predict(x_valid_df_pca)

# Print the obtained accuracy score after applying SVC approach
print("Accuracy Score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

# Make predictions on the test dataset
Prediction_SVC_PCA = classifier.predict(x_test_df_pca)

Accuracy Score:  0.9586666666666667


## **Random Forest Classifier Model**
This is a classifier model that combines multiple decision trees to make predictions. It is known for its high accuracy and robustness against overfitting.

Here, the **Random Forest Classifier** model is built on the initial dataframe before feature selection. The accuracy score obtained **(0.8533)** is **much lower** than the value obtained from **Support Vector Classifier** model **(0.94)**. So, **Random Forest Classifier is not the best model for this label.**

In [ ]:
# Perform Random Forest Classifier model on the initial dataframe
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(x_train_df, y_train_df)
y_valid_pred = classifier.predict(x_valid_df)

# Print the obtained accuracy score after applying Random Forest approach
print("Accuracy Score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

# Make predictions on the test dataset
Prediction_RF = classifier.predict(x_test_df)

Accuracy Score:  0.8533333333333334


Here, the **Random Forest Classifier** model is built on the dataframe after feature selection through PCA approach. The accuracy score obtained **(0.804)** is **much lower** compared to the value obtained from **Support Vector Classifier** model after feature selection **(0.9453)**. So, **Random Forest Classifier is not the best model for this label.**

In [ ]:
# Perform Random Forest Classifier model on the dataframe obtained after performing PCA Approach
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(x_train_df_pca, y_train_df)
y_valid_pred = classifier.predict(x_valid_df_pca)

# Print the obtained accuracy score after applying Random Forest approach
print("Accuracy Score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

# Make predictions on the test dataset
Prediction_RF_PCA = classifier.predict(x_test_df_pca)

Accuracy Score:  0.804


## **Output CSV File Generation**
The initial accuracy score is **(0.94)**. Best model chosen is **Support Vector Classifier Model with best hyper parameters**. Because the accuracy score obtained here **(0.9586)** is **higher** than the score obtained from other models under various criteria. Therefore, predictions made by this model is used to create the output CSV file for submission.

In [ ]:
# Create the output CSV file
output_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/ML Project/Layer 9/solutions.csv')
output_df['label_1'] = Prediction_SVC_PCA
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/CS4622 - Machine Learning/ML Project/Layer 9/solutions.csv',index=False)